## Import Required Libraries

In [4]:
import statistics
import matplotlib.pyplot as plt
from matplotlib.cbook import flatten
import numpy as np
import random
from scipy import stats
from scipy.spatial import ConvexHull
from scipy.stats import t
from statsmodels.distributions.empirical_distribution import ECDF

## UU-test

In [5]:
def ks(X,alpha=0.01):
    """
    Kolmogorov-Smirnov test for uniformity.
    
    Parameters:
    X : array-like
        Sample data.
    alpha : float, optional
        Significance level (default is 0.01).
    
    Returns:
    int
        1 if the null hypothesis (that X follows a uniform distribution) is not rejected at level alpha.
        0 otherwise.

    """

    # If all values in X are the same (min(X) == max(X)), the function returns 1.
    if min(X) != max(X):
        np.seterr(invalid='ignore')
        ktest = stats.kstest(X, cdf='uniform', args=(min(X), max(X) - min(X)), N=len(X))        
        pvalue = ktest[1]           
        return int(pvalue > alpha)
    else:
        return 1

In [6]:
def compute_gcmlcm(X, plot_on = False):  
    """
    Computes the greatest convex minorant (GCM) and least concave majorant (LCM) 
    of the empirical cumulative distribution function (ECDF) for a given dataset.
    
    Parameters:
    X : array-like
        Input sample data (numerical values).
    plot_on : bool, optional
        If True, generates a histogram and plots the convex hull visualization (default is False).
    
    Returns:
    gcm : list
        List of x-values corresponding to the greatest convex minorant (GCM).
    lcm : list
        List of x-values corresponding to the least concave majorant (LCM).
    
    Raises:
    ValueError:
        If `X` contains fewer than 3 points, as GCM and LCM cannot be computed.
    
    Notes:
    - The function computes the ECDF of `X` and constructs its convex hull.
    - The vertices of the convex hull are used to determine GCM and LCM points.
    - The first ECDF value is removed to avoid trivial cases.
    - If `plot_on=True`, the function visualizes the dataset histogram and convex hull construction.
    """

    if len(X)<3:
        print("Error!!!! Not enough points to compute gcm and lcm points!! (Need at least 3!)") 
        raise ValueError("Not enough points")
    else: 
        X = sorted(X)  
        ecdf = ECDF(X)
        F, x = ecdf.y,  ecdf.x
        F = F[1:]
        x = x[1:]         
        points = np.array(list(zip(x, F)))
        K = ConvexHull(points)        
        Kx, gcm, lcm = [], [], []   
        Kx = K.vertices.tolist()
     
    ind_0 = Kx.index(0)
    ind_last = Kx.index(len(F)-1)
    if ind_0 > ind_last:
        lcm_ind = sorted(Kx[ind_last:ind_0])
        gcm_ind = sorted([i for i in Kx if i not in lcm_ind])
    else:
        gcm_ind = sorted(Kx[ind_0:ind_last])
        lcm_ind = sorted([i for i in Kx if i not in gcm_ind])
    gcm = sorted([X[i] for i in gcm_ind])
    lcm = sorted([X[i] for i in lcm_ind]) 
    
    if plot_on:
        plt.figure(figsize=(12,4))
        plt.subplot(1,2,1)
        plt.hist(X, bins=50)        
        plt.subplot(1,2,2)
        plt.plot(x,F,'b-')
        plt.plot(x[gcm_ind], F[gcm_ind],'r*', mfc='none', label="gcm")
        plt.plot(x[lcm_ind], F[lcm_ind],'go', mfc='none', label="lcm")        
        plt.plot(x[gcm_ind], F[gcm_ind],'r--')
        plt.plot(x[lcm_ind], F[lcm_ind],'g--')        
        plt.plot([x[-1], max(x[gcm_ind])], [F[-1], max(F[gcm_ind])],'r--')
        plt.plot([x[0], min(x[lcm_ind])], [F[0], min(F[lcm_ind])],'g--')
        plt.legend()
        plt.show()
    return gcm, lcm

In [7]:
def Backward_search(PB,eR,X):
    """
    Performs a backward search to refine a partition boundary.

    The function iteratively removes elements from the partition boundary (PB) 
    and checks whether the remaining partition is still uniform 
    using the Kolmogorov-Smirnov (KS) test. If a valid partition is found, 
    the boundary is extended to eR, and success is marked.

    Parameters:
    PB : list
        A list representing partition boundaries.
    eR : float
        The right endpoint of the partition being tested (is stable).
    X : array-like
        The dataset on which the partitioning is applied.

    Returns:
    PB : list
        The updated partition boundaries after backward search.
    success : int
        1 if a valid partition is found, 0 otherwise.

    Notes:
    - The function starts by removing the last element of PB.
    - It iterates while PB is not empty, checking if the data subset 
      between `max(PB)` and `eR` satisfies the KS test (`ks(xx)`).
    - If a valid partition is found, `eR` is added back, and success is set to 1.
    - If no valid partition is found, PB is returned empty.
    """
    success = 0
    del PB[-1]
    while PB:
        xx = [x for x in X if max(PB) <= x <= eR]
        if ks(xx):
            PB.append(eR)
            success = 1           
            return PB, success
        del PB[-1]      
    return PB, success

In [8]:
def Forward_search(PF,eL,X):   
    """
    Performs a forward search to refine a partition boundary.

    The function iterates through the partition boundaries (PF) and finds the first 
    valid boundary satisfying the Kolmogorov-Smirnov (KS) test. If a valid partition 
    is found, the corresponding boundary is returned along with a success flag.

    Parameters:
    PF : list
        A list representing partition boundaries.
    eL : float
        The left endpoint of the partition being tested.
    X : array-like
        The dataset on which the partitioning is applied.

    Returns:
    PF_ : float or empty list
        The selected partition boundary if a valid one is found, otherwise an empty list.
    success : int
        1 if a valid partition is found, 0 otherwise.
    """
    success = 0    
    for point in PF:        
        if point > eL:
            xx = [x for x in X if eL <= x <= point]        
            if ks(xx):
                PF_ = point
                success = 1                
                return PF_, success  
    if not success:
        PF_ = []
    
    return PF_, success 

In [9]:
def consistent(gcm, lcm): 
    """
    This function organizes the GCM and LCM points into consistent subsets.
    If only one of the sets is provided, it returns that set as a single consistent subset.
    If GCM points appear before LCM points without overlap, they are merged into one ordered set.
    Otherwise, the function (`determine_consistent_subsets`) is called to separate them into multiple consistent subsets.

    Parameters:
    gcm : list
        A list of x-values corresponding to the greatest convex minorant (GCM).
    lcm : list
        A list of x-values corresponding to the least concave majorant (LCM).

    Returns:
    C : list of lists
        A list containing one or more subsets of consistent points.
    num_of_cons_sets : int
        The number of consistent subsets identified.
    msg : list of tuples
        A list of messages indicating the structure of the returned subsets (type of points).
        - If only LCM exists, msg = [('1', [])]
        - If only GCM exists, msg = [('0', [])]
        - If GCM and LCM are strictly ordered, msg = [('01', len(gcm))]
        - Otherwise, msg is determined by `determine_consistent_subsets`.

    Notes:
    - If both `gcm` and `lcm` exist but are non-overlapping (i.e., `max(gcm) <= min(lcm)`), they are combined into a single sequence.
    - Otherwise, `determine_consistent_subsets(gcm, lcm)` is used to find consistent groupings.
    """
    C=[[]]
    ind=[[],[]]
    num_of_cons_sets = 1
    
    # no gcm points or lcm starts first
    if not gcm:
        C[0] = lcm
        ind[0]=[1]*len(lcm)
        msg=[('1',[])]
    elif not lcm:
        C[0] = gcm
        ind[0]=[0]*len(gcm)
        msg=[('0',[])]
    elif max(gcm)<=min(lcm):
        C[0] = gcm+lcm
        ind[0] = [0]*len(gcm) + [1]*len(lcm)
        msg=[('01',len(gcm))]
    else:
        C,num_of_cons_sets,msg = determine_consistent_subsets(gcm,lcm)  
    return C, num_of_cons_sets, msg

In [10]:
def determine_consistent_subsets(gcm,lcm):
    """
    Creates consistent subsets of the ordered gcm+lcm list.
    
    The function takes two lists of points, `gcm` (greatest convex minorant) and `lcm`
    (least concave majorant), sorts them together, and partitions them into separate 
    consistent subsets by removing lcm points existing between gcm points.
    
    Parameters:
    gcm : list
        A list of x-values corresponding to the greatest convex minorant (GCM).
    lcm : list
        A list of x-values corresponding to the least concave majorant (LCM).
    
    Returns:
    C : list of lists
        A list containing one or more subsets of consistent points.
    num_of_cons_sets : int
        The number of consistent subsets identified.
    msg : list of lists
        A list indicating the start index of LCM in each subset.
    """
    gcmlcm = sorted(gcm+lcm)
    type_ = list(map(lambda x: 1 if x in lcm else 0, gcmlcm))
    ind_last_0s = [i for i in range(len(type_)-1) if type_[i] == 0 and type_[i+1]==1]
    num_of_cons_sets = len(ind_last_0s)
    C=[[] for _ in range(num_of_cons_sets)]
    
    msg = [['01',0] for i in range(num_of_cons_sets)]
    
    for j in range(num_of_cons_sets):
        for i in range(len(type_)):
            if i<=ind_last_0s[j]:
                if type_[i]!=1:
                    C[j].append(gcmlcm[i])
            else:
                if type_[i]!=0:
                    C[j].append(gcmlcm[i])
        msg[j][1] = min([i for i in range(len(C[j])) if C[j][i] in lcm])
    return C, num_of_cons_sets, msg

In [11]:
def sufficient(P,X):   
    """
    Checks if a partition P is sufficient for the dataset X and refines it if necessary.

    Parameters:
    P : list
        List of partition points.
    X : list
        List of data points.

    Returns:
    P_ : list
        A refined partition satisfying the sufficiency condition.
    success : int
        1 if a sufficient partition is found, 0 otherwise.
    notUU : list
        Intervals where sufficiency fails.
    """
    notUU=[]; success = 1
    xx = [x for x in X if P[0] <= x <= P[-1]] 
    if ks(xx):     
        P_=[P[0],P[-1]]
        return P_, success, notUU
    eL = P[0]
    P_ = [eL]   
    
    while max(P_) != max(P):
        eR = min([x for x in P if x > max(P_)])
        xx = [x for x in X if eL <= x <= eR]
        if ks(xx):
            P_.append(eR)
        else:
            PF, success = Forward_search(P, eL,X)
            if success:
                P_.append(PF)
            else:
                PB, success = Backward_search(P_, eR , X)
                if not success:
                    #
                    notUU.append([eL, eR])
                    P_.append(eR)
                else:
                    P_ = PB
        eL = max(P_)
    if notUU:
        P_ = []
        success = 0
    return P_, success, notUU   

In [12]:
def UU(SG,PI,SL,X,plot_on, detect_peaks):
    
    """
    Recursive function to create a piecewise linear unimodal approximation (PL_S(X)) of the ecdf of X

    Parameters:
    ----------
    SG : list
        List of gcm points
    PI : list
        Middle part (last gcm to first lcm)
    SL : list
        List of lcm points
    X : list
        The dataset (assumed to be a 1D list of numerical values).
    plot_on : bool
        If True, enables visualization.
    detect_peaks : bool
        If True, it aims to find the peak of the data (works for unimodal datasets only!)
        Then no recursion calls occur for the middle part, and returns the middle part unchanged.
        The mean value of the middle part can be used to compute the peak.

    Returns:
    -------
    SG_ : list
        Updated list of gcm points, where each segment is uniform.
    PI_ : list
        Updated middle part.
    SL_ : list
        Updated list of lcm points, where each segment is uniform.
    success : int
        Indicator of success (1 if approximation was found, 0 otherwise).
    notUU : list
        List of segments that were found to be **not unimodal/not uniform**.
    """
    
    X = sorted(X)    
    SG_ = SG
    SL_ = SL
    notUU, PG, PL = [], [], []
    xx = [x for x in X if PI[0] <= x <= PI[1]]    
    if ks(xx):        
        success = 1
        return SG_,PI,SL_,success,notUU
    
    gcm, lcm = compute_gcmlcm(xx,plot_on)
    C, num_of_cons_sets, msg = consistent(gcm,lcm)

    if not num_of_cons_sets:
        success = 1       
        PI_ = [min(X),max(X)]
        SG_ = SG_ + []
        SL_ = SL_ + []
        
    for i in range(num_of_cons_sets):
        type_of_GL, ind_of1 = msg[i]
        
        if type_of_GL == '01':
            PG=C[i][:ind_of1]
            PL=C[i][ind_of1:]
            PI_=[max(PG),min(PL)]
            PG_, success, notUU_ = sufficient(PG,X)
  
            if not success:
                notUU.extend([x + y for x, y in zip(notUU_, [[0]]*len(notUU_))])
                continue       
            PL_, success, notUU_ = sufficient(PL,X)
      
            if not success:
                notUU.extend([x + y for x, y in zip(notUU_, [[1]]*len(notUU_))])
                continue
            SG_= SG_ + PG_
            SL_ = SL_ + PL_
            
        elif type_of_GL == '0': 
            PG = C[i][0]
            PI_ = []
            PG_, success, notUU_ = sufficient(PG,X)
       
            if not success:
                notUU.extend([x + y for x, y in zip(notUU_, [[0]]*len(notUU_))])
                continue             
            SG_= SG_ + PG_
            
        else:
            PI_ = []
            PL = C[i][0]
            PL_, success, notUU_ = sufficient(PL,X)
           
            if not success:
                notUU.extend([x + y for x, y in zip(notUU_, [[1]]*len(notUU_))])
                continue         
            SL_ = SL_ + PL_

        if success:
            if PI_:
                if detect_peaks: return SG_,PI_,SL_,success,notUU
                SG_,PI_,SL_,success,notUU_ = UU(SG_,PI_,SL_,X,plot_on, detect_peaks)

                if not success:
                    notUU.extend(notUU_)
                    continue
            return SG_,PI_,SL_,success,notUU
        
    if not success:
        PI_, SG_, SL_ = [], [], []        
         
    return SG_,PI_,SL_,success,notUU               

In [13]:
def UUtest(X, plot_on = False, detect_peaks = False):   
    """
    Wrapper function for the UU method to create a piecewise linear unimodal approximation of the ecdf of X.

    Parameters:
    ----------
    X : list
        The dataset (assumed to be a 1D list of numerical values).
    plot_on : bool, optional
        If True, enables visualization.
    detect_peaks : bool, optional
        If True, it detects the peak of X (working on unimodal datasets only!).

    Returns:
    -------
    S : list
        Sorted list of segment boundaries (unimodal partitions).
    notUU : list
        List of segments that were found to be not uniform.
    success : int
        Indicator of success (1 if approximation was found, 0 otherwise).
    PI_ : list
        Middle part of the unimodal piecewise linear approximation if `detect_peaks` is enabled, otherwise an empty list.
    """
    
    if not X:
        print("Empty dataset") 
        raise ValueError("Empty dataset")
    X=sorted(X) 
#     X = add_unif_noise(X)
#     print(f'X={X}')
    SG, SL = [], []    
    PI=[min(X), max(X)]  
    SG_,PI_,SL_,success,notUU = UU(SG,PI,SL,X,plot_on, detect_peaks)
    S=sorted(list(set(SG_ + PI_ + SL_)))
    if success: notUU = []
    if detect_peaks: return S, notUU, success, PI_
    return S, notUU, success, []

In [14]:
# def add_unif_noise(f):
# add uniform noise into the data (e.g., in case of discrete values)
# # X1 = X[:,0].tolist()
#     from collections import Counter
#     cnt = Counter(f)
#     data_to_change = [k for k, v in cnt.items() if v > 1]
#     if not data_to_change:
#         return f
#     sumi=0
#     for i in range(len(f)):
#         if f[i] in data_to_change:        
#             noise = np.random.uniform(-0.01,0.01,1)
#             f[i] += noise[0]
#             sumi+=1
#     return f



## Functions for Uniform Mixture Modeling (UMM) and Sampling

In [15]:
def calculate_percentage_in_interval(X, S):
    """
    Computes the proportion of data points that fall within each interval defined in S.

    Parameters:
    ----------
    X : list
        The dataset (assumed to be a sorted 1D list of numerical values).
    S : list
        A sorted list of boundary points (of the determined PL_S(X)) defining intervals.

    Returns:
    -------
    p : list
        A list of percentages representing the proportion of X in each interval.
    """
    if len(S) == 1: return [1]
    count = []
    for i in range(len(S)-1):
        count.append(len([x for x in X if S[i] < x <= S[i+1]]))
    count[0] = count[0] + 1
    p = [x / len(X) for x in count]
    return p

def fitUU(X):
    """
    Fits a unimodal piecewise linear cdf to the dataset's ecdf.

    Parameters:
    ----------
    X : list
        The dataset (assumed to be a 1D list of numerical values).

    Returns:
    -------
    component_point_ratio : list
        A list containing the proportion of points belonging to each component
        (here: 1 unimodal component (to do: multiple unimodal components)
    interval_point_ratio : list
        A list of lists, where each sublist contains the proportion of points in each interval.
    intervals_per_component : list
        A list containing the interval boundaries for each detected unimodal component.
    component_ranges : list
        A list containing the global range [min(X), max(X)].
    """
    intervals_per_component = []
    S, _ , _, _ = UUtest(X)
    if S:
        #print('unimodal')
        component_point_ratio = [1]
        interval_point_ratio = [calculate_percentage_in_interval(X,S)]
        intervals_per_component.append(S)
        component_ranges = [min(X), max(X)]
    else:
        print("The dataset is multimodal. No fit is achieved.")
    return component_point_ratio, interval_point_ratio, intervals_per_component, component_ranges
    
def locate_point_in_interval(x, ranges):   
    """
    Locates which interval in 'ranges' the given point 'x' falls into.

    Parameters:
    ----------
    x : float
        The data point to locate.
    ranges : list
        A sorted list of boundary points defining intervals.

    Returns:
    -------
    index : int
        The index of the interval in which 'x' falls.
        Returns -2 if 'x' is below the lowest interval.
        Returns -1 if 'x' is above the highest interval.
    """
    if x < ranges[0]:
        return -2            
    elif x > ranges[-1]:
        return -1                
    for i in range(len(ranges)-1):        
        if ranges[i] <= x <= ranges[i+1]:           
            return i       

def cdfUU(X, S, p):
    """
    Computes the cumulative distribution function (CDF) for a unimodal distribution.

    Parameters:
    ----------
    X : list
        The dataset (assumed to be a sorted 1D list of numerical values).
    S : list
        A sorted list of interval boundaries.
    p : list
        The proportion of points in each interval.

    Returns:
    -------
    y : list
        The CDF values corresponding to the sorted dataset X.
    X : list
        The sorted dataset.
    """
    X = sorted(X)    
    y = []
    for i in range(len(X)):
        ind = locate_point_in_interval(X[i], S)   
        if ind == -2:           
            y.append(0)
        elif ind == -1:
            y.append(1)          
        else: 
            y_val = sum(p[:ind]) + p[ind] * (X[i]-S[ind]) / (S[ind+1]-S[ind])               
            y.append(y_val)        
    return y, X


def pdfUU(X, component_point_ratio, interval_point_ratio, intervals_per_component, component_ranges):
    """
    Computes the probability density function (PDF) for a given unimodal distribution.

    Parameters:
    ----------
    X : list
        The dataset (assumed to be a sorted 1D list of numerical values).
    component_point_ratio : list
        Proportion of points in each component.
    interval_point_ratio : list
        A list of proportions of points in each interval.
    intervals_per_component : list
        A list containing the interval boundaries for each detected unimodal component.
    component_ranges : list
        The global range of the dataset.

    Returns:
    -------
    pdf_val : list
        The computed PDF values for each point in X.
    X : list
        The sorted dataset.
    """
    X = sorted(X)
    pdf_val = []
    point_ind = [locate_point_in_interval(x, component_ranges) for x in X]
    for i in range(len(X)):
        if point_ind[i] < 0:            
            pdf_val.append(0)
            continue
            
        pdfUU_of_x = pdfUU_of_point(X[i], intervals_per_component[point_ind[i]], interval_point_ratio[point_ind[i]])        
        pdf_val.append(component_point_ratio[point_ind[i]] * pdfUU_of_x)
    return pdf_val, X


def pdfUU_of_point(x, S, p):
    """
    Computes the probability density function (PDF) value for a single point 'x'.

    Parameters:
    ----------
    x : float
        The data point for which to compute the PDF.
    S : list
        A sorted list of interval boundaries.
    p : list
        The proportion of points in each interval.

    Returns:
    -------
    pdf_val : float
        The computed PDF value for the given point 'x'.
    """
    i = locate_point_in_interval(x, S)
    if i<0: return 0
    pdf_val = p[i] / (S[i+1] - S[i])
    return pdf_val


def sample(X, n):
    """
    Generates 'n' samples from the estimated unimodal distribution.

    Parameters:
    ----------
    X : list
        The dataset (assumed to be a sorted 1D list of numerical values).
    n : int
        The number of samples to generate.

    Returns:
    -------
    samples : list
        A list of generated samples following the estimated unimodal distribution.
    """
    samples = []
    component_point_ratio, interval_point_ratio, intervals_per_component, _ = fitUU(X)
    for j in range(len(component_point_ratio)):    
        random_val = np.random.multinomial(round(component_point_ratio[j] * n), interval_point_ratio[j])
        S = intervals_per_component[j]    
        for i in range(len(S)-1):
            random_num = [(S[i+1]-S[i]) * random.random() for _ in range(random_val[i])]
            samples.extend([S[i] + x for x in random_num])
    return samples

## Demo - Examples

In [22]:
def run_Demo(dip_test = True, UMM = True, generate_sample = True):


    # Unimodal case
    X = np.random.normal(0, 1, 1500)

    ## Multimodal case (uncomment)
    # X1 = np.random.normal(0, 1, 1000)
    # X2 = np.random.normal(5,1,500)
    # X3 = np.random.uniform(15,20,700)
    # X = np.concatenate((X1, X2, X3))
    
    
    # Run the unimodality test
    X = X.tolist()
    S, notUU, success, _ = UUtest(X, plot_on = True, detect_peaks = False)
    
    if success:
        print("* A piecewise linear approximation of the ecdf of your dataset has been successfully determined.\n")
        print("* Your dataset is unimodal.\n")
        print(f"* The points that define the approximation cdf are saved in S\n\n S: {S}\n")
    
        # For detecting the peak
        _, _, _, PI_ = UUtest(X, plot_on = False, detect_peaks = True)
        # mean or median value can be used
        peak = np.mean(PI_)
        print(f"The peak of your dataset is: {peak}\n")
    
    else:
        notUU_indicator = {0: "gcm", 1:"lcm"}
        print("* No unimodal piecewise linear approximation of the ecdf of your dataset can be determined.\n")
        print("* Your dataset is multimodal.\n")
        print("* The cause of multimodality is the segment/segments in notUU, i.e.,\n")
        for segment in notUU:
            print(f"segment: {segment[:2]} defined by '{notUU_indicator[segment[2]]}' points.\n")


        # Compare with Hartigans' dip-test
        # pip install diptest
    if dip_test:    
        import diptest
        dip, pval = diptest.diptest(np.array(X))
        print(f"Hartigans' dip test p-value = {pval}")

    if success:    
        if UMM:        
            # Fit the unimodal model (only applicable if the dataset is unimodal)
                component_point_ratio, interval_point_ratio, intervals_per_component, component_ranges = fitUU(X)
            
            # Compute and plot the pdf and cdf of the PL_S approximation            
                plt.figure(figsize=(16,4))
                plt.subplot(1,3,1)
                pdf_val, X = pdfUU(X, component_point_ratio, interval_point_ratio, intervals_per_component, component_ranges)
                plt.hist(X, bins=50, density = True)
                plt.plot(X, pdf_val, 'r', label="UMM")
                plt.title("Unimodal Mixture Model (UMM)")
                plt.legend()
                
                ecdf = ECDF(X)
                plt.subplot(1,3,2)
                cdf_val, X= cdfUU(X,S,interval_point_ratio[0])
                plt.plot(X, cdf_val, 'r', label='cdfUU')
                plt.plot(ecdf.x, ecdf.y, c='b', label='ecdf')
                plt.legend(['cdf','ecdf'])
                
                plt.subplot(1,3,3)
                cdf_val, S= cdfUU(S,S,interval_point_ratio[0])
                plt.plot(ecdf.x, ecdf.y, c='b')
                plt.plot(S, cdf_val, 'ro--')
                plt.legend(['ecdf', 'cdf'])
                plt.show()
    
        # Generate a sample from the original dataset X following the same unimodal distribution 
        if generate_sample:
                
            size = 1000
            data_sample = sample(X, size)
            
            plt.figure(figsize=(10,8))
            plt.subplot(2,2,1)
            plt.hist(X, bins=50)
            plt.title("Original")
            
            plt.subplot(2,2,3)
            ecdf = ECDF(X)
            plt.plot(ecdf.x, ecdf.y, c='b', label='ecdf')
            
            plt.subplot(2,2,2)
            plt.hist(data_sample, bins=50)
            plt.title("Sample")
            
            plt.subplot(2,2,4)
            ecdf = ECDF(data_sample)
            plt.plot(ecdf.x, ecdf.y, c='m', label='ecdf')
            
            plt.show()

In [ ]:
if __name__ == "__main__":
    run_Demo()